In [4]:
!pip install vacances-scolaires-france
!pip install jours-feries-france

  Preparing metadata (setup.py) ... done
  Created wheel for jours-feries-france: filename=jours_feries_france-0.7.0-py3-none-any.whl size=4997 sha256=d7a69c8e7249a18a7a6b393a5c907f618d946b5257cc6ea66a9ada2c40dced74
  Stored in directory: /Users/roybensimon/Library/Caches/pip/wheels/6c/91/19/bc9f25ded8d3f5b48cfa3a01cfce9cf767fc027988e4eee9fb
Successfully built jours-feries-france


In [13]:
import pathlib as Path
import pandas as pd
import numpy as np
from vacances_scolaires_france import SchoolHolidayDates
from jours_feries_france import JoursFeries
from datetime import datetime

# j'ouvre le external data d'origine
data = pd.read_csv("external_data.csv")
data
  

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,hnuage1,nnuage2,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4
0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,96,...,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,98,...,1500.0,2.0,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,98,...,480.0,4.0,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN
3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,97,...,1740.0,3.0,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN
4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,82,...,330.0,4.0,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,7149,2020-09-30 09:00:00,101540,-30,8,230,4.4,289.95,286.85,82,...,400.0,7.0,6.0,2200.0,NaN,NaN,NaN,NaN,NaN,NaN
3318,7149,2020-09-30 12:00:00,101320,-210,8,190,4.9,292.05,285.55,66,...,870.0,7.0,6.0,1900.0,NaN,NaN,NaN,NaN,NaN,NaN
3319,7149,2020-09-30 15:00:00,101140,-180,7,190,4.1,291.55,286.45,72,...,820.0,7.0,6.0,2200.0,NaN,NaN,NaN,NaN,NaN,NaN
3320,7149,2020-09-30 18:00:00,101020,-130,6,190,2.7,290.15,285.25,73,...,2160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:


def data_creation(data):
    
    df_original = data
    df_external = pd.read_csv("external_data.csv")

    # on crée les colonnes correspondant aux periode de confinement
    df_external["quarantine1"] = np.where(
    (df_external["date"] <= "2020-12-15") & (df_external["date"] >= "2020-10-30"), "True", "False")
    df_external["quarantine2"] = np.where(
    (df_external["date"] <= "2021-05-03") & (df_external["date"] >= "2021-04-03"), "True", "False")


    #on crée deux variables, holiday and bank_holiday pour savoir si une donnée correspond a un jour férié ou un jour de vacances
    hol = SchoolHolidayDates()

    df_external['holiday'] = df_external.apply(
    lambda row: hol.is_holiday_for_zone(datetime.strptime(row["date"], '%Y-%m-%d %H:%M:%S').date(), 'C'), axis=1)

    df_external['bank_holiday'] = df_external.apply(
    lambda row: JoursFeries.is_bank_holiday(datetime.strptime(row["date"], '%Y-%m-%d %H:%M:%S').date(), zone="Métropole"), axis=1)


    n_rows = df_external.shape[0]

    #  proportion de missingvalues pour chaque colonne
    proportion_missing_values = df_external.isnull().sum() / n_rows

    # on supprime les colonnes avec un seuil de 15% de valeurs manquantes
    columns_to_drop = proportion_missing_values[proportion_missing_values >= 0.15].index

    # on supprime les colonnes sélectionnées du DataFrame ext
    df_external = df_external.drop(columns=columns_to_drop)

    col_to_delete_2 = ["numer_sta", "cod_tend","pmer","nbas", "rr1" ,"rr3","rr12","rr6","rr24", "nbas", "ht_neige", "rafper","ff", "per", "w2", "td"]

    df_external = df_external.drop(columns=col_to_delete_2)
    
    # Je emplace les valeurs manquantes par la médiane
    df_external = df_external.fillna(df_external.median(numeric_only=True))

    df_external['date'] = pd.to_datetime(df_external['date'], errors='coerce')
    
    df_original['date'] = pd.to_datetime(df_original['date'], errors='coerce')
    df_external['date'] = pd.to_datetime(df_external['date'], errors='coerce')

    df_merge = pd.merge_asof(df_original.sort_values("date"), df_external.sort_values("date"), on="date")
    
    return df_merge
    

In [33]:
print(data_creation(data))

      numer_sta                date    pmer  tend_x  cod_tend  dd_x    ff  \
0          7149 2020-09-01 00:00:00  102050     -10         8   340   1.6   
1          7149 2020-09-01 03:00:00  101990     -60         6   290   1.1   
2          7149 2020-09-01 06:00:00  102000      10         3   360   1.8   
3          7149 2020-09-01 09:00:00  101970       0         5   360   2.9   
4          7149 2020-09-01 12:00:00  101850    -110         6    40   2.6   
...         ...                 ...     ...     ...       ...   ...   ...   
3317       7149 2021-10-21 00:00:00   99800      50         0   230  11.8   
3318       7149 2021-10-21 03:00:00  100270     460         3   260  14.6   
3319       7149 2021-10-21 06:00:00  101000     720         1   240   7.7   
3320       7149 2021-10-21 09:00:00  101230     230         1   240   7.0   
3321       7149 2021-10-21 12:00:00  101160     -60         8   240   4.6   

         t_x      td  u_x  ...  raf10_y  etat_sol_y  ssfrai_y  perssfrai_y 

In [34]:
new_data_external = data_creation(data)

In [35]:
new_data_external

,numer_sta,date,pmer,tend_x,cod_tend,dd_x,ff,t_x,td,u_x,...,raf10_y,etat_sol_y,ssfrai_y,perssfrai_y,nnuage1_y,hnuage1_y,quarantine1_y,quarantine2_y,holiday,bank_holiday
0,7149,2020-09-01 00:00:00,102050,-10,8,340,1.6,285.75,282.55,81,...,2.4,0.0,0.0,-60.0,3.0,1080.0,False,False,False,False
1,7149,2020-09-01 03:00:00,101990,-60,6,290,1.1,283.95,282.05,88,...,1.5,0.0,0.0,-30.0,3.0,1080.0,False,False,False,False
2,7149,2020-09-01 06:00:00,102000,10,3,360,1.8,284.25,282.85,91,...,5.0,0.0,0.0,-60.0,1.0,4000.0,False,False,False,False
3,7149,2020-09-01 09:00:00,101970,0,5,360,2.9,291.25,283.35,60,...,5.0,0.0,0.0,-30.0,1.0,950.0,False,False,False,False
4,7149,2020-09-01 12:00:00,101850,-110,6,40,2.6,293.95,281.25,44,...,6.7,0.0,0.0,-60.0,1.0,1410.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,7149,2021-10-21 00:00:00,99800,50,0,230,11.8,287.05,281.45,69,...,22.5,0.0,0.0,-60.0,5.0,1020.0,False,False,False,False
3318,7149,2021-10-21 03:00:00,100270,460,3,260,14.6,286.05,279.35,64,...,26.2,0.0,0.0,-30.0,7.0,980.0,False,False,False,False
3319,7149,2021-10-21 06:00:00,101000,720,1,240,7.7,284.05,279.95,76,...,11.0,0.0,0.0,-60.0,3.0,1100.0,False,False,False,False
3320,7149,2021-10-21 09:00:00,101230,230,1,240,7.0,286.05,280.75,70,...,11.1,0.0,0.0,-30.0,1.0,700.0,False,False,False,False


In [38]:
new_data_external.to_csv("new_data_external.csv", index=False)
